<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین پنجم (تحلیل لینک)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        دسترسی به داده‌ها و مدل‌های ذخیره شده
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        به دلیل حجم بالای فایل‌های ذخیره شده، از قرار دادن آن‌ها در پوشه نوتبوک صرف نظر کرده و با اجرای قطعه کد زیر، از طریق گوگل درایو به آن‌ها دسترسی پیدا می‌کنیم.
    </p>
</div>

In [1]:
# import sys
# from google.colab import drive
# drive.mount('/content/drive')
# address = "University/Term 6/MIR/Homeworks/HW3"
# sys.path.append(f"/content/drive/My Drive/{address}")
# %cd /content/drive/My\ Drive/$address

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [2]:
! pip install bs4
! pip install tqdm
! pip install pandas
! pip install requests
! pip install hazm
! pip install unidecode
! pip install datasets
! pip install nltk
! pip install networkx

In [3]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import pickle
import zipfile
import requests
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
from pathlib import Path
from datasets import Dataset
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٦٨ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [4]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

CATEGORIES_CLASSES = {
    'Politics': 0,
    'World': 1,
    'Economy': 2,
    'Society': 3,
    'City': 4,
    'Sport': 5,
    'Science': 6,
    'Culture': 7,
    'IT': 8,
    'LifeSkills': 9,
}

CLASSES_CATEGORIES = {v: k for k, v in CATEGORIES_CLASSES.items()}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [5]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [6]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [7]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    os.remove('dataset.csv')
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [8]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [9]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [10]:
def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


def get_preprocessed_texts(dataset, preprocessor, mode, save=False):
    preprocessed_texts = []
    if mode == 'process':
        texts = [data_to_text(data) for _, data in dataset.iterrows()]
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    if mode == 'load':
        preprocessed_texts = load_preprocessed_texts()
    if save:
        save_preprocessed_texts(preprocessed_texts)
    return preprocessed_texts

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم، یا داده‌های پیش‌پردازش‌شده‌ی ذخیره‌شده را از فایل مربوطه بازیابی می‌کنیم.
    </p>
</div>

In [11]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')

In [12]:
preprocessed_texts = get_preprocessed_texts(dataset, preprocessor, mode='load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
         با اجرای قطعه کد زیر، یک مجموعه داده‌ی کوچک‌تر از روی مجموعه داده‌ی اصلی ایجاد می‌کنیم.
    </p>
</div>

In [13]:
def get_mini_dataset(len_each_category=1000):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    return mini_dataset


# Almost 10K News
mini_dataset = get_mini_dataset()

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٣. رتبه‌بندی و تحلیل لینک
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
در این قسمت، با استفاده از دو روش PageRank و HITS به رتبه‌بندی اخبار می‌پردازیم. در قسمت اول ابتدا ماتریس مجاورت را برای اخبار دیتاست می‌سازیم.
    </p>
</div>

<div>
    <h4 style='direction:rtl;text-align:justify;'>
       ۱. ساخت ماتریس مجاورت
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای ساخت ماتریس مجاورت به این صورت عمل می‌کنیم که هر خبر را به عنوان یک راس در نظر می‌گیریم. سپس با در نظر گرفتن تعداد کلمات مشترک هر دو خبر، اگر این تعداد از یک threshold مشخص بیش‌تر باشد، آن دو خبر را به یک‌دیگر وصل کرده و تعداد کلمات مشترک را به عنوان وزن آن یال دو طرفه لحاظ می‌کنیم. البته، در انتها، نرمال‌سازی نیز به صورت خودکار انجام خواهد شد.
    </p>
</div>

In [14]:
THRESHOLD = 2

def common_words(bow1, bow2):
    bow1 = set(bow1)
    bow2 = set(bow2)
    same_words = bow1.intersection(bow2)
    if len(same_words) >= THRESHOLD:
        return len(same_words)
    else:
        return 0

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، عناوین خبری مجموعه‌ی داده‌ها را پیش‌پردازش کرده و به عنوان رئوس گراف در نظر می‌گیریم.
    </p>
</div>

In [15]:
def get_adj_matrix(dataset):
    titles = dataset['title'].to_list()
    titles = [preprocessor.preprocess(title) for title in titles]
    adjacency_matrix = np.zeros(shape=(dataset.shape[0], dataset.shape[0]))
    for i in tqdm(range(dataset.shape[0]), position=0, leave=True):
        for j in range(i + 1, dataset.shape[0]):
            weight = common_words(titles[i], titles[j])
            adjacency_matrix[i, j] = weight
            adjacency_matrix[j, i] = weight
    return adjacency_matrix

In [16]:
adjacency_matrix = get_adj_matrix(mini_dataset)

100%|██████████| 9437/9437 [01:35<00:00, 98.89it/s]  


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، گراف مبتنی بر کتاب‌خانه‌ی networkx از روی ماتریس مجاورتی که در قبل به دست آمد، ساخته می‌شود.
    </p>
</div>

In [17]:
graph = nx.from_numpy_array(adjacency_matrix)

<div>
    <h4 style='direction:rtl;text-align:justify;'>
       ٢. اجرای الگوریتم‌های PageRank و HITS
    </h4>
</div>

In [18]:
def get_top_rankings(dataset, graph, method, k=5):
    if method == 'PageRank':
        pr = nx.pagerank(graph, alpha=0.9)
        vals = np.array(list(pr.values()))
        idx = np.argsort(-vals)
        for i, title in enumerate(dataset.iloc[idx[:k]]['title']):
            print(f"Rank {i + 1} with pr = {pr[idx[i]]:.8f}")
            print(title)
            print()
    elif method == 'HITS':
        h, a = nx.hits(graph)
        vals_h = np.array(list(h.values()))
        idx_h = np.argsort(-vals_h)
        vals_a = np.array(list(a.values()))
        idx_a = np.argsort(-vals_a)
        print("---------------------------------------- Hubs ----------------------------------------")
        for i, title in enumerate(dataset.iloc[idx_h[:k]]['title']):
            print(f"Rank {i + 1} with hub value = {h[idx_h[i]]:.8f}")
            print(title)
            print()
        print()
        print("------------------------------------- Authorities ------------------------------------")
        for i, title in enumerate(dataset.iloc[idx_a[:k]]['title']):
            print(f"Rank {i + 1} with authority value = {a[idx_a[i]]:.8f}")
            print(title)
            print()
    else:
        print("Not valid method!")

In [101]:
get_top_rankings(mini_dataset, graph, method='PageRank')

Rank 1 with pr = 0.00106249
ببینید | اوکراین؛ از آژیر حمله هوایی تا آتش سوزی در فرودگاه | روسیه: این جنگ نیست! | زلنسکی وضعیت نظامی اعلام کرد | واکنش آمریکا، انگلیس و ترکیه

Rank 2 with pr = 0.00094111
فردا ۱/۵ میلیون دوز واکسن کرونا به ایران می‌رسد | جزئیات واکسیناسیون گروه‌های سنی در جنوب و شرق کشور

Rank 3 with pr = 0.00085860
آمار تفکیکی واکسیناسیون کرونا در ایران تا ۲۳ شهریور | ۱۲ میلیون نفر هردو دوز واکسن را زدند

Rank 4 with pr = 0.00085610
۵ واکسن ایرانی کرونا به مرحله نهایی رسیدند | نیاز به ۱۲۰ میلیون دوز برای واکسیناسیون ۷۰ درصد جمعیت ایران

Rank 5 with pr = 0.00084152
زمان ارسال لایحه بودجه ۱۴۰۱ شهرداری تهران به شورای شهر اعلام شد | قوانین و مقررات شهرسازی دست و پای مردم را بسته است



<div>
    <h4 style='direction:rtl;text-align:justify;'>
       تحلیل خروجی الگوریتم PageRank
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در اولین نگاه، نکته‌ای که توجه‌مان را جلب می‌کند تعداد کلمه‌ی عنوان محوری‎ترین خبرها است. این را انتظار داشتیم؛ چرا که با توجه به این که وزن یال‌های ورودی و خروجی یک راس متناسب با تعداد کلمات مشترک عنوان خبر آن راس با راس دیگر است، عناوین طولانی‌تر شانس بیش‌تری برای داشتن اشتراک با عناوین دیگر دارند. حال، کمی دقیق‌تر نگاه کنیم:
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        مهم‌ترین خبری که رنک اول را دارد، ویژگی‌های جالبی دارد. اسامی پنج کشور اوکراین، روسیه، آمریکا، انگلیس و ترکیه در آن وجود دارد. به وضوح می‌توان حدس زد که یکی از عوامل مهم بودن این خبر همین است! چرا که در تمام دسته‌های مختلف این امکان هست که راجع به این کشورهای مهم صحبت شود؛ علی الخصوص کشورهای اوکراین، روسیه و آمریکا که تعداد بسیاری از خبرهای چند ماه گذشته را به خود اختصاص می‌دهند. نکته‌ی دیگر این است که با توجه به به‌روز بودن مجموعه‌ی داده‌های ما، خبرهای مربوط به جنگ‌های اخیر در آن زیاد است؛ پس، انتظار داریم کلمات کلیدی‌ای مانند "جنگ"، "نظامی"، "حمله" و حتی "آتش" بر روی ایجاد ارتباط این خبر با خبرهای دیگر موثر باشد.
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        هم‌چنین، می‌توان دید که مهم‌ترین خبرهای بعدی‌ مربوط به کرونا و واکسیناسون هستند. این نیز بسیار مورد انتظار است. توجه داشته باشید که خبرهای کراول شده‌ی ما همگی پس از دوره کرونا هستند و داغ‌‌ترین خبرهای مربوط به کرونا نیز مربوط به واکسیناسیون انجام شده داخل ایران می‌باشند. همین کلمات کلیدی‌ و پرتکرار "کرونا"، "واکسن"، "واکسیناسیون"، "دوز" و حتی خود "ایران" می‌توانند روی رنک این خبرها اثر به‌سزایی داشته باشند.
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        مشابه مواردی که گفته شد، کلمات مربوط به تهران و شهرداری آن نیز در خبرهای ما بسیار است. علی الخصوص که یک دسته با عنوان "City" شامل همین جنس خبرها است. به همین دلیل دور از انتظار نیست که رنک پنجم مربوط به یک خبر مهم از همین دسته باشد.
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        توجه کنید که اختلاف بین pr خبرهای رنک با یک‌دیگر بسیار کم – چیزی در حدود یک‌ صدهزارم – است. پس، ممکن است با توجه به این اختلاف ناچیز تعداد دیگری از خبرهای مهم در رنک‌های بعدی قرار گرفته باشند و این با توجه به ماهیت PageRank یک امر طبیعی است.
    </p>
</div>

In [124]:
get_top_rankings(mini_dataset, graph, method='HITS')

c:\users\mohammad\appdata\local\programs\python\python38\lib\site-packages\networkx\algorithms\link_analysis\hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


---------------------------------------- Hubs ----------------------------------------
Rank 1 with hub value = 0.00812863
نرخ یورو و  ۲۸ ارز دیگر افزایش یافت | جدیدترین قیمت رسمی ارزها در ۹ مرداد ۱۴۰۰

Rank 2 with hub value = 0.00798444
قیمت رسمی دلار به کانال ۲۷ هزار تومان نزدیک شد | جدیدترین قیمت ارزها در یکم شهریور ۱۴۰۰

Rank 3 with hub value = 0.00797153
کاهش قیمت طلا و سکه در بازار | جدیدترین نرخ طلا و سکه در ۲۲ خرداد ۱۴۰۰

Rank 4 with hub value = 0.00796374
نرخ ۲۰ ارز افزایش یافت | جدیدترین قیمت رسمی ارزها در ۱۲ خرداد ۱۴۰۰

Rank 5 with hub value = 0.00785182
نرخ ۲۸ ارز کاهش یافت | جدیدترین قیمت رسمی ارزها در ۱۷شهریور ۱۴۰۰


------------------------------------- Authorities ------------------------------------
Rank 1 with authority value = 0.00812863
نرخ یورو و  ۲۸ ارز دیگر افزایش یافت | جدیدترین قیمت رسمی ارزها در ۹ مرداد ۱۴۰۰

Rank 2 with authority value = 0.00798444
قیمت رسمی دلار به کانال ۲۷ هزار تومان نزدیک شد | جدیدترین قیمت ارزها در یکم شهریور ۱۴۰۰

Rank 3 with authority va

<div>
    <h4 style='direction:rtl;text-align:justify;'>
       تحلیل خروجی الگوریتم HITS
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با توجه به مجموعه‌ی داده‌های ما و نحوه‌ی عملکرد روش HITS این خروجی به شدت انتظار می‌رفت. به قول یک بزرگی، اقتصاد همه چیز است!
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در خبرهای کراول شده‌ی ما بخش حائز اهمیتی را خبرهای اقتصادی تشکیل می‌دهند و از این بخش قابل توجه، اکثر اخبار مربوط به نوسانات و رویدادهای مربوط به بازار ارزها، طلا و سکه است. خصوصاً که یک سری خبر روزانه در این مورد منتشر می‌شود که امور اقتصادی مربوط به این‌ها را گزارش می‌دهد. این جنس خبرها که در خروجی آمده‌اند، بسیار به یک دیگر شبیه هستند؛ حتی قالب نوشتاری آن‌ها نیز بسیار مشابه یک‌دیگر است. بنابراین در گراف ارتباطات خبری ما، یک زیرگراف چگال مربوط به این خبرها داریم که تشکیل یک زیرگراف حدوداً کامل را داده‌اند؛ مثل یک مهمانی که در آن همه‌ی مهمان‌ها به یک‌دیگر دست می‌دهند! به همین دلیل است که هم در بخش Hub و هم در بخش Authority خبرهای حدوداً یکسانی را می‌بینیم؛ این خبرها به یک‌دیگر ارجاع می‌دهند و می‌گیرند.
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        شاید این سوال پیش بیاید که چرا در بخش PageRank این گونه خبرها را ندیدیم؟ پاسخ بسیار ساده‌ای برای این سوال وجود دارد. توجه کنید که در الگوریتم PageRank همواره مجموع وزن نهایی یال‌های خروجی از هر راس برابر با یک است؛ به عبارت دیگر، نرمال‌سازی وزن‌ها را داریم. این در حالی است که در روش HITS به تعداد یال‌های خروجی از هر راس اهمیت ویژه‌ای می‌دهیم. برای خبرهای اقتصادی مذکور، این موضوع به روشنی دیده می‌شود، ولی به دلیل نرمال‌سازی در روش PageRank این ویژگی کم‌اثر خواهد بود.
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        از این که وقت ارزشمند خود را برای بررسی این نوتبوک صرف کرده‌اید، صمیمانه سپاسگزاریم. (:
    </p>
</div>